In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
import os

# Configurazione
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'dataset')
MODEL_DIR = os.getcwd()

TEST_PATH = os.path.join(DATA_DIR, 'test_step6.csv')
MODEL_PATH = os.path.join(MODEL_DIR, 'severe_model_stacking_production.pkl')


In [2]:
# Caricamento
df_test = pd.read_csv(TEST_PATH)
model_dict = joblib.load(MODEL_PATH)
print("Dizionario modello caricato. Keys:", model_dict.keys())

# Estrazione Componenti
rf_model = model_dict['rf_model']
xgb_model = model_dict['xgb_model']
lgbm_model = model_dict['lgbm_model']
meta_model = model_dict['meta_model']
threshold = model_dict.get('threshold', 0.5)
features_order = model_dict.get('features_order', None)

print(f"Threshold salvato: {threshold}")

Dizionario modello caricato. Keys: dict_keys(['rf_model', 'xgb_model', 'lgbm_model', 'meta_model', 'threshold', 'features_order'])
Threshold salvato: 0.5999999999999999


In [3]:
# Preparazione Dati
target_col = 'IS_SEVERE'
X_test = df_test.drop(columns=[target_col])
y_test = df_test[target_col]

# Allineamento Features
if features_order is not None:
    # Check missing
    missing = set(features_order) - set(X_test.columns)
    for c in missing:
        X_test[c] = 0
    X_test = X_test[features_order]
    print("Features allineate secondo 'features_order'.")
else:
    print("ATTENZIONE: 'features_order' non trovato. Uso l'ordine del dataframe.")

Features allineate secondo 'features_order'.


In [4]:
# Infeernza Base Models
print("Generazione predizioni base models...")
p_rf = rf_model.predict_proba(X_test)[:, 1]
p_xgb = xgb_model.predict_proba(X_test)[:, 1]
p_lgbm = lgbm_model.predict_proba(X_test)[:, 1]

# Stacking
# L'ordine deve essere coerente con come è stato trainato il meta model.
# Cerchiamo di dedurlo dai feature_names_in_ del meta_model se possibile
X_meta = None
if hasattr(meta_model, 'feature_names_in_'):
    meta_features = meta_model.feature_names_in_
    print("Features attese dal meta model:", meta_features)
    # Creiamo un dataframe per essere sicuri
    df_meta = pd.DataFrame({
        'rf': p_rf, # Possibili nomi, da verificare
        'xgb': p_xgb,
        'lgbm': p_lgbm,
        'RF': p_rf,
        'XGB': p_xgb,
        'LGBM': p_lgbm
    })
    # Proviamo a selezionare quelle che matchano
    # Spesso i nomi sono generici o 'rf_pred', ecc. 
    # Se fallisce, usiamo l'ordine standard [rf, xgb, lgbm]
    try:
         X_meta_df = pd.DataFrame()
         for feat in meta_features:
             # Euristica base per mappare nomi features
             if 'rf' in feat.lower(): X_meta_df[feat] = p_rf
             elif 'xgb' in feat.lower(): X_meta_df[feat] = p_xgb
             elif 'lgb' in feat.lower(): X_meta_df[feat] = p_lgbm
         X_meta = X_meta_df
    except:
        X_meta = np.column_stack((p_rf, p_xgb, p_lgbm))   
else:
    # Fallback ordine standard
    X_meta = np.column_stack((p_rf, p_xgb, p_lgbm))

# Se X_meta è vuoto o ha colonne sbagliate (succede se euristica fallisce), fallback
if isinstance(X_meta, pd.DataFrame) and X_meta.shape[1] != 3:
    print("Euristica nomi fallita, uso ordine sequenziale [rf, xgb, lgbm]")
    X_meta = np.column_stack((p_rf, p_xgb, p_lgbm))
elif X_meta is None:
     X_meta = np.column_stack((p_rf, p_xgb, p_lgbm))


# Predizione Finale
y_prob = meta_model.predict_proba(X_meta)[:, 1]
y_pred = (y_prob >= threshold).astype(int)

print("Predizione stacking completata.")

Generazione predizioni base models...


Features attese dal meta model: ['rf_prob' 'xgb_prob' 'lgbm_prob']
Predizione stacking completata.


In [5]:
# Metriche
print("="*50)
print(" REPORT STACKING MODEL")
print("="*50)
print(f"ACCURACY:  {accuracy_score(y_test, y_pred):.4f}")
print(f"PRECISION: {precision_score(y_test, y_pred):.4f}")
print(f"RECALL:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-SCORE:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_prob):.4f}")
print("-"*50)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

 REPORT STACKING MODEL
ACCURACY:  0.8050
PRECISION: 0.3797
RECALL:    0.8180
F1-SCORE:  0.5186
ROC-AUC:   0.8910
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.80      0.88    146090
           1       0.38      0.82      0.52     21526

    accuracy                           0.81    167616
   macro avg       0.67      0.81      0.70    167616
weighted avg       0.89      0.81      0.83    167616

TP: 17608, FP: 28766, TN: 117324, FN: 3918
